# Мэтчинг товаров дилера и заказчика

## Описание проекта

Заказчик производит несколько сотен различных товаров бытовой и промышленной химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь, занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках. Для оценки ситуации, управления ценами и бизнесом в целом, заказчик периодически собирает информацию о том, как дилеры продают их товар. Для этого они парсят сайты дилеров, а затем сопоставляют товары и цены. Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт заказчик. Например, могут добавляться новый слова (“универсальный”, “эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с товарами производителя делается вручную.  
Основная идея - предлагать несколько товаров заказчика, которые с наибольшей вероятностью соответствуют размечаемому товару дилера. Предлагается реализовать это решение, как онлайн сервис, открываемый в веб- браузере. Выбор наиболее вероятных подсказок делается методами машинного обучения

**ЦЕЛЬ:** разработать решения, которое автоматизирует процесс сопоставления товаров.  

**Задачи:**
   - выгрузить данные
   - сделать предобработку
   - обработать текст в столбцах с названиями товаров
   - создать ембеддинги предложений 
   - попробовать разные модели,
   - оценить метрики и выбрать лучшую модель


## Описание данных

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые данные:  

1. `dealers` - список дилеров:
   - id - уникальный ключ дилера;
   - name - наименование дилера</br>
</br>  
2. `dealer_products` - результат работы парсера площадок дилеров:
   - product_key - уникальный номер позиции;
   - price - цена;
   - product_url - адрес страницы, откуда собраны данные;
   - product_name - заголовок продаваемого товара;
   - date - дата получения информации;
   - dealer_id - идентификатор дилера (внешний ключ к dealers)</br>
</br>
3. `products` - список товаров, которые производит и распространяет заказчик:
   - id - уникальный ключ товара в базе заказчика
   - article - артикул товара;
   - ean_13 - код товара (см. EAN 13)
   - name - название товара;
   - cost - стоимость;
   - recommended_price - рекомендованная цена;
   - category_id - категория товара;
   - ozon_name - названиет товара на Озоне;
   - name_1c - название товара в 1C;
   - wb_name - название товара на Wildberries;
   - ozon_article - описание для Озон;
   - wb_article - артикул для Wildberries;
   - ym_article - артикул для Яндекс.Маркета;</br>  
</br>  
4. `match` - таблица матчинга товаров заказчика и товаров дилеров:
   - key - внешний ключ к dealer_products;
   - product_id - внешний ключ к products;
   - dealer_id - внешний ключ к dealers.

In [377]:
import nltk
import spacy
import re 
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from pymystem3 import Mystem
#nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import sparse
from scipy.spatial.distance import pdist
import warnings
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 500)

## Выгрузка и обработка данных

In [378]:
# Выгрузим данные из 4х таблиц в отдельные датафреймы
dealers = pd.read_csv('marketing_dealer.csv', sep=';')
dealer_products = pd.read_csv('marketing_dealerprice.csv', sep=';')
products = pd.read_csv('marketing_product.csv', sep=';')
match = pd.read_csv('marketing_productdealerkey.csv', sep=';')

In [379]:
dealers.sample(5)

,id,name
9,12,sdvor
5,7,Komus
13,16,Vimos
11,14,VegosM
14,4,Baucenter


In [380]:
dealers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18 non-null     int64 
 1   name    18 non-null     object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


In [381]:
dealers['name'].unique()

array(['Moi_vibor_WB', 'Akson', 'Bafus', 'Castorama', 'Cubatora', 'Komus',
       'Megastroy', 'OnlineTrade', 'Petrovich', 'sdvor', 'simaLand',
       'VegosM', 'Vse_instrumeni', 'Vimos', 'Baucenter', 'Leroy_Merlin',
       'Мasterstroy_spb_OZON\r\n', 'Unicleaner_OZON'], dtype=object)

В таблице `dealers` собрана информация по названиям дилеров и их id. Всего в таблице представлено 18 уникальных дилеров.  
Для решения поставленной задачи на текущий момент данная таблица не требуется.

In [382]:
dealer_products.sample(5)

,id,product_key,price,product_url,product_name,date,dealer_id
8207,8263,15881549,585.0,https://www.vseinstrumenti.ru/product/konditsioner-dlya-belya-s-aromatom-yaponskogo-chaya-prosept-crystal-rinser-5l-254-5-957298/,Кондиционер для белья с ароматом японского чая PROSEPT Crystal Rinser 5л 254-5,2023-07-17,15
6612,6698,20621584,888.0,https://www.vseinstrumenti.ru/product/ognebiozaschita-dlya-drevesiny-prosept-2-gruppa-bestsvetnyj-gotovyj-sostav-10-kg-064-10-2186988/,"Огнебиозащита для древесины PROSEPT 2 группа, бесцветный, готовый состав, 10 кг 064-10",2023-07-14,15
17461,17610,200544252,613.0,https://www.bafus.ru/200544252/,"Просепт Professional Duty Universal удалитель клейкой ленты, клея, наклеек (400 мл)",2023-07-28,3
16360,16399,26390925,565.0,https://vimos.ru/product/antiseptik-lessiruusij-dekorativnyj-prosept-biolasur-sosna-09l,Антисептик лессирующий декоративный Prosept Biolasur сосна 0.9л,2023-07-26,16
1560,2756,108893,198.0,https://www.sdvor.com/ekb/product/sredstvo-dlja-chistki-kovrov-i-tekstilnyh-izdelij-carpet-dryclean-500ml-prosept-108893,Средство для чистки ковров и текстильных изделий Prosept Carpet DryClean 500 мл,2023-07-12,12


In [383]:
# удалим лишний столбец 'id
dealer_products = dealer_products.drop(['id'], axis=1)

In [384]:
dealer_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   20416 non-null  object 
 1   price         20416 non-null  float64
 2   product_url   20182 non-null  object 
 3   product_name  20416 non-null  object 
 4   date          20416 non-null  object 
 5   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 957.1+ KB


In [385]:
dealer_products.nunique()

product_key     1965
price           1286
product_url     1883
product_name    1953
date              14
dealer_id         18
dtype: int64

In [386]:
sorted(dealer_products['date'].unique())

['2023-07-11',
 '2023-07-12',
 '2023-07-13',
 '2023-07-14',
 '2023-07-17',
 '2023-07-18',
 '2023-07-19',
 '2023-07-21',
 '2023-07-24',
 '2023-07-25',
 '2023-07-26',
 '2023-07-27',
 '2023-07-28',
 '2023-07-31']

В таблице из 20416 записей лишь около 10% уникальных ключей, ссылок и названий продуктов. Все данные собраны за 14 дней: с 11-07 по 31-0-23.

In [387]:
dealer_products.isna().sum()

product_key       0
price             0
product_url     234
product_name      0
date              0
dealer_id         0
dtype: int64

В столбце product_url имеются пропуски, исследуем его подробнее.

In [388]:
no_url = dealer_products[dealer_products['product_url'].isna()]['dealer_id'].unique()
no_url

array([7], dtype=int64)

In [389]:
dealers[dealers['id'] == no_url[0]]

,id,name
5,7,Komus


In [390]:
dealer_products[dealer_products['dealer_id'] == 7]

,product_key,price,product_url,product_name,date,dealer_id
97,1462352,189.0,NaN,Средство для удаления жира и нагара Prosept Cooky Grill 500 мл,2023-07-11,7
177,1462335,233.0,NaN,Полироль для мебели Prosept Universal Polish 500 мл,2023-07-31,7
397,1462337,213.0,NaN,Средство для чистки каминных стекол Prosept Universal Hard 500 мл,2023-07-11,7
689,1565304,149.0,NaN,Средство для мытья пола Prosept Multipower 800 мл,2023-07-11,7
717,1462346,159.0,NaN,Средство для прочистки труб Prosept Bath Prof жидкость 1 л,2023-07-11,7
...,...,...,...,...,...,...
19213,1462347,189.0,NaN,Средство для сантехники Prosept Bath Acryl +акрил 1 л,2023-07-31,7
19214,1462346,159.0,NaN,Средство для прочистки труб Prosept Bath Prof жидкость 1 л,2023-07-31,7
19215,1462352,189.0,NaN,Средство для удаления жира и нагара Prosept Cooky Grill 500 мл,2023-07-31,7
19216,1462354,219.0,NaN,Средство для чистки ковровых покрытий Prosept Carpet DryClean шампунь 500 мл,2023-07-31,7


Все имеющиеся в таблице пропуски относятся к дилеру под номером 7 - Комус.

In [391]:
# Проверим длину названий продуктов для определения неявных пропусков.
dealer_products['product_name'].str.len().min(), dealer_products['product_name'].str.len().max()

(8, 131)

In [392]:
dealer_products[dealer_products['product_name'].str.len() == 8]

,product_key,price,product_url,product_name,date,dealer_id
32,44231946,994.0,https://www.wildberries.ru/catalog/44231946,ОSB BASE,2023-07-11,1
1718,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
1885,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
1963,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-11,1
3276,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-12,1
5110,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-13,1
6766,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-14,1
8346,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-17,1
11054,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-19,1
12269,44231946,994.0,https://www.wildberries.ru/catalog/44231946/detail.aspx?targetUrl=SP,ОSB BASE,2023-07-21,1


In [393]:
# проверим данные на дубликаты
dealer_products.duplicated().sum()

726

In [394]:
duplicates = dealer_products.duplicated()
dealer_products[duplicates].sort_values(by='product_name').head(20)

,product_key,price,product_url,product_name,date,dealer_id
1945,30420470,263.0,https://www.wildberries.ru/catalog/30420470/detail.aspx?targetUrl=SP,Bath Acid,2023-07-11,1
1867,30420470,263.0,https://www.wildberries.ru/catalog/30420470/detail.aspx?targetUrl=SP,Bath Acid,2023-07-11,1
1866,44231972,325.0,https://www.wildberries.ru/catalog/44231972/detail.aspx?targetUrl=SP,Bath Acid + Концентрат 1 л,2023-07-11,1
1944,44231972,325.0,https://www.wildberries.ru/catalog/44231972/detail.aspx?targetUrl=SP,Bath Acid + Концентрат 1 л,2023-07-11,1
1849,44232028,271.0,https://www.wildberries.ru/catalog/44232028/detail.aspx?targetUrl=SP,Bath Acid Концентрат,2023-07-11,1
1927,44232028,271.0,https://www.wildberries.ru/catalog/44232028/detail.aspx?targetUrl=SP,Bath Acid Концентрат,2023-07-11,1
1978,44231954,325.0,https://www.wildberries.ru/catalog/44231954/detail.aspx?targetUrl=SP,Bath Acid Концентрат 1 л,2023-07-11,1
1900,44231954,325.0,https://www.wildberries.ru/catalog/44231954/detail.aspx?targetUrl=SP,Bath Acid Концентрат 1 л,2023-07-11,1
1985,44232073,868.0,https://www.wildberries.ru/catalog/44232073/detail.aspx?targetUrl=SP,Bath Acid Концентрат 5 л,2023-07-11,1
1907,44232073,868.0,https://www.wildberries.ru/catalog/44232073/detail.aspx?targetUrl=SP,Bath Acid Концентрат 5 л,2023-07-11,1


In [395]:
# уберём дубликаты по столбцам: ключ, url и названию
dealer_products.drop_duplicates(subset=['product_key', 'product_url', 'product_name'], inplace=True)

In [396]:
dealer_products.duplicated(subset=['product_key', 'product_url', 'product_name']).sum()

0

In [397]:
# сбросим индексы
dealer_products.reset_index(drop = True, inplace = True)

В таблице `dealer_products` 20416 записей.  
Имеются пропуски в столбце `product_url` - 234 записи и все для дилера с id 7 - Komus. Полных дублей в таблице нет, но есть повторяющиейся записи в зависимости от даты выгрузки. 
Все столбцы имеют правильный тип, кроме даты, в рамках проекта дату приводить к нужному формату нет необходимости.  
Столбец `product_key` содержит данные в текстового типа, он состоит из ключей не только в виде числа, но и в виде ссылок на сайты с продуктами.   

Столбец `product_name` является целевым: по нему будем находить соответствие продуктов из базы заказчика.  
В названиях имеются как слова на кириллице, так и на латинице; есть специальные символы, единицы измерения разные: кг, л, мл; в некоторых названиях в конце указан код, состоящий из цифр и "-"; попадаются сокращения (например: дер. конструкций, д/удаления), в рамках одного названия встречаются буквы в разных регистрах.

In [398]:
# удалим лишний столбец 'Unnamed: 0'
products = products.drop(['Unnamed: 0'], axis=1)

In [399]:
products.sample(5)

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
39,508,М003-2,4.610093e+12,"Набор для посудомоечной посуды PROSEPT, 2 средства (гель и ополаскиватель)",512.00,1195.0,NaN,"Набор для посудомоечной посуды PROSEPT, 2 средства (гель и ополаскиватель)","Набор для посудомоечной машины PROSEPT, 2 средства","Набор для посудомоечной посуды Splash, гель и ополаскиватель",463666869.0,150666268.0,М003-2,NaN
96,498,М036-2,4.610093e+12,"Антисептик ULTRA, концентрат, 1 л, 2 шт",720.00,1645.0,NaN,"Антисептик ULTRA, концентрат, 1 л, 2 шт","Антисептик ULTRA, концентрат, 1 л, 2 шт","Антисептик ULTRA, концентрат, 1 л, 2 шт",NaN,NaN,NaN,NaN
177,49,109-075,4.680008e+12,"Средство для удаления ржавчины и минеральных отложений щадящего действияBath Acid концентрат 1:200-1:500 / 0,75 л",71.81,168.0,52.0,"Средство для удаления ржавчины и минеральных отложений щадящего действияBath Acid концентрат 1:200-1:500 / 0,75 л","Средство для удаления ржавчины и минеральных отложений щадящего действияBath Acid концентрат 1:200-1:500 / 0,75 л","Средство для удаления ржавчины и минеральных отложений щадящего действияBath Acid концентрат 1:200-1:500 / 0,75 л",NaN,NaN,NaN,NaN
456,80,266-5,4.680008e+12,Ополаскиватель для пароконвектоматов с режимом автомат. ОчисткиSplash Shine концентрат / 5 л,477.96,1116.0,40.0,"Ополаскиватель для пароконвектоматов PROSEPT Splash Shine, 5 л.","Ополаскиватель для пароконвектоматов PROSEPT Splash Shine, 5 л.","Ополаскиватель для пароконвектоматов PROSEPT Splash Shine, 5 л.",451659369.0,149705853.0,266-5,NaN
443,54,113-5,4.680008e+12,Средство усиленного действия для удаления ржавчины и минеральных отложений Bath Acid + концентрат 1:200-1:500 / 5 л,385.00,900.0,52.0,"Усиленное средство для удаления ржавчины и минеральных отложений PROSEPT Bath Acid Plus, 5 л.","Усиленное средство для удаления ржавчины и минеральных отложений PROSEPT Bath Acid Plus, 5 л.","Усиленное средство для удаления ржавчины и минеральных отложений PROSEPT Bath Acid Plus, 5 л.",413264559.0,149811048.0,113-5,NaN


In [400]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   ean_13             464 non-null    float64
 3   name               494 non-null    object 
 4   cost               491 non-null    float64
 5   recommended_price  491 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         340 non-null    float64
 12  ym_article         337 non-null    object 
 13  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(1), object(7)
memory usage: 54.4+ KB


In [401]:
products.nunique()

id                   496
article              496
ean_13               464
name                 487
cost                 338
recommended_price    319
category_id           38
ozon_name            454
name_1c              473
wb_name              451
ozon_article         365
wb_article           339
ym_article           337
wb_article_td         32
dtype: int64

In [402]:
# удалим столбец wb_article_td так как он содержит мало записей и не содержит важной информации.
products.drop(['wb_article_td'], axis=1, inplace=True)

In [403]:
products.isna().sum()

id                     0
article                0
ean_13                32
name                   2
cost                   5
recommended_price      5
category_id           49
ozon_name             38
name_1c               11
wb_name               41
ozon_article         131
wb_article           156
ym_article           159
dtype: int64

In [404]:
mask = products['name'].isna()
products[mask]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
23,503,0024-7 о,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150126213.0,NaN
35,504,w022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [407]:
#удалим строки, где пропуски в названиии товара
products.dropna(subset=['name'], inplace=True)
products.reset_index(drop=True, inplace= True)

In [409]:
mask = products['cost'].isna()
products[mask]

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б
107,449,0024-06 м12,NaN,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN,NaN,25.0,NaN,"Герметик акриловый цвет Медовый 0,6 л (12 шт)",NaN,NaN,NaN,NaN
108,454,0024-06 о12,NaN,"Герметик акриловый цвет Орех, ф/п 600мл (12 штук )",NaN,NaN,25.0,NaN,"Герметик акриловый цвет сосна, ф/п 600мл (12 штук )",NaN,NaN,NaN,NaN


In [410]:
products.duplicated().sum()

0

In [411]:
# изучим наименования продуктов детальнее
products[['name', 'name_1c']].sample(5)

,name,name_1c
474,"Герметик акриловый цвет Сосна, 7 кг","Герметик акриловый цвет Сосна, 7 кг"
453,"Чистящий крем для кухниCooky Universal ""ледяная свежесть""готовый состав / 0,5 л","Чистящий крем для кухни PROSEPT Cooky Universal ""ледяная свежесть"", 500 мл."
406,Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л,"Удалитель цемента PROSEPT CEMENT CLEANER, концентрат, 1л."
423,"ОГНЕБИОЗАЩИТА для древесины 2 группа, красный готовый состав / 20 кг","PROSEPT огнебиозащита с цветовой индикацией. 2-ая группа, 20 л."
400,Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 5 л,"Антисептик невымываемый PROSEPT ULTRA, коричневый, концентрат, 5 л."


На первый взгляд в названиях из 1С меньше лишней или технической информации, меньше опечаток.

In [412]:
# проверим минимальную и максимальную длину названия
products['name'].str.len().min(), products['name'].str.len().max()

(3, 136)

In [413]:
products[products['name'].str.len() == 3]
# в данной строке отсутствует название, запись можно удалить

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article
96,436,Р1 09005,4.680008e+12,,500.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [414]:
i = products[products['name'].str.len() == 3].index
products.drop(i, inplace = True)
products.reset_index(drop = True, inplace = True)

In [416]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 493 non-null    int64  
 1   article            493 non-null    object 
 2   ean_13             463 non-null    float64
 3   name               493 non-null    object 
 4   cost               490 non-null    float64
 5   recommended_price  490 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         339 non-null    float64
 12  ym_article         337 non-null    object 
dtypes: float64(6), int64(1), object(6)
memory usage: 50.2+ KB


В таблице `products` 496 записей.  
Имеются 2 записи, где отсутствует большая часть информации.  
Для товаров *Герметик акриловой цвет Белый, 7 кг; Герметик акриловый цвет Медовый 0,6 л (12 шт); Герметик акриловый цвет Орех, ф/п 600мл (12 штук)* отсутствуют стоимость и рекомендованная цена.
Дубликаты отсутствуют.  

Для построения модели мэчинга можем использовать данные в столбце `name` или `name_1c`, всего имеется 487 уникальных наименований.
В названиях имеются опечатки, лишние пробелы, специальные символы, иногда отсутствуют пробелы между словами: часто сливаются слова на кириллице и латинице. В части продуктов указана рекомендуемая концентрация, для некоторых продуктов указан вес (в кг.), а для других объём (в мл. или л.). Концентрация, вес или количество обычно указываются в конце названия. В рамках одного названия встречаются буквы в разных регистрах. Максимальная длина наименования продукта 136 символов, минимальная - 30.

In [417]:
# удалим лишний столбец 'Unnamed: 0'
match = match.drop(['id'], axis = 1)

In [418]:
match.sample(5)

,key,dealer_id,product_id
985,26391427,16,312
848,7226107,13,121
1533,717999700,17,155
989,23295827,16,298
938,26391007,16,354


In [419]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 40.0+ KB


In [420]:
match.nunique()

key           1700
dealer_id       18
product_id     438
dtype: int64

In [421]:
match.sort_values(by='key', ascending=False)

,key,dealer_id,product_id
440,https://kub02.ru/catalog/prosept/otbelivatel_dlya_drevesiny_prosept_50_1_1_1l/,6,240
439,https://kub02.ru/catalog/prosept/maslo_dlya_zashchity_polkov_prosept_sauna_oil_gotovyy_sostav_0_25l/,6,320
434,https://kub02.ru/catalog/prosept/lak_dlya_bani_i_sauny_termostoykiy_akrilovyy_prosept_0_9l/,6,321
449,https://kub02.ru/catalog/prosept/antiseptik_universalnyy_protiv_gribka_i_pleseni_prosept_antiplesen_got_sostav_5l/,6,290
432,https://kub02.ru/catalog/prosept/antiseptik_universalnyy_dlya_vnutr_i_naruzhn_prosept_universal_1l/,6,259
...,...,...,...
383,100067710,3,291
224,100067709,3,289
175,100067708,3,397
170,100067707,3,242


In [422]:
match.duplicated().sum()

0

В таблице `match` 1700 записей, кол-во уникальных id дилеров совпадает с количеством в таблице `dealers`. Пропусков нет, дубликатов тоже. Столбец `key` имеет текстовый формат, в нём есть записи не только уникальных ключей, но и url продуктов.  
Данная таблица может пригодиться, когда будем оценивать эффективность мэтчинга.

## Предобработка названий

In [423]:
# заполним пропуски в name_1c данными из столбца name
products['name_1c'].fillna(products['name'], inplace=True)

В дальнейшем будем работать со столбцами `1c_name` из таблицы `products` и `product_name` из `dealer_products`.
Необходимо произвести предобработку текста, чтобы названия в обеих таблицах были наиболее схожи, для этого необходимо:
1. убрать лишние пробелы
2. привести к нижнему регистру
3. добавить пробелы между русскими словами и английскими: как до, так и после
4. убрать концентрацию, оставить только объём/вес 
5. перевести всё к одним ед. измерения: литры - в миллилитры (в идеале)

In [424]:
# функция для базовой обработки текста
def clean_text(text):
    #добавляем пробелы между русскими и английскими словами
    pattern = re.compile(r'(?<=[а-яА-Я])(?=[A-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])')
    text = re.sub(pattern, ' ', text)
    #убираем указание концентрации
    pattern2 = re.compile(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*')
    text = re.sub(pattern2, '', text)
    #убираем специальные символы
    remove = string.punctuation
    #remove = remove.replace("-", "") # не убираем дефисы
    text = re.sub('[%s]' % re.escape(remove), ' ', text)
    # убираем лишние пробелы между словами
    text = re.sub(r'\s+', ' ', text)
    #приводим все слова к нижнему регистру
    text = text.lower()
    return text

# функция убирает служебные слова и лемматизирует текст
def preprocess_text(text):
    # удаление одиноко стоящих слов
    text = re.sub(r'\s+[a-zA-Zа-яА-Я0-9]\s+', ' ', text)
    russian_stopwords = stopwords.words("russian")
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
#     mystem = Mystem() 
#     tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

# функция выделеяет единицы измерения из текста
def extract_measure(text):
    measurements = []
    pattern = r'\b(\d+)\s?[л|мл|кг]+'
    match = re.search(pattern, text)
    if match:
        measurements = match.group(1)
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    else:
        measurements = 0
        text = text.replace(pattern, '')
        text = text.replace(' ', '')
    return measurements

# функция вовращает список длин
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

In [425]:
# создадим столбцы с единицbами измерения товара в обеих таблицах
products['measures'] = products['name_1c'].apply(extract_measure)
products['measures'] = products['measures'].astype(int)

dealer_products['measures'] = dealer_products['product_name'].apply(extract_measure)
dealer_products['measures'] = dealer_products['measures'].astype(int)

In [426]:
#создадим столбцы с длиной названия товара
products['name_length'] = get_text_length(np.array(products['name_1c']))
dealer_products['name_length'] = get_text_length(np.array(dealer_products['product_name']))

In [427]:
%%time
# создадим новый столбец marketing_name - он включает в себя все названия из 1с
products['marketing_name'] = products['name_1c'].apply(clean_text)
pattern = r'\b(\d+)\s?[л|мл|кг]+'
products['marketing_name'] = products['marketing_name'].str.replace(pattern, '')

CPU times: total: 297 ms
Wall time: 296 ms


In [429]:
products[['name_1c','marketing_name']].sample(3,random_state=1)

,name_1c,marketing_name
354,"Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.",антисептик универсальный prosept хмф бф гост
107,"Герметик акриловый цвет сосна, ф/п 600мл (12 штук )",герметик акриловый цвет сосна ф п 12 штук
165,Гель эконом-класса для мытья посуды вручную. Без запахаCooky Е концентрированное средство / 5 л ПЭТ,гель эконом класса для мытья посуды вручную без запаха cooky е концентрированное средство пэт


In [430]:
%%time
# лемматизируем текста
nlp = spacy.load("ru_core_news_lg")
products['marketing_name'] = products['marketing_name'].apply(preprocess_text)

CPU times: total: 6.06 s
Wall time: 6.13 s


In [431]:
products[['name_1c','marketing_name']].sample(3,random_state=1)

,name_1c,marketing_name
354,"Антисептик универсальный PROSEPT ХМФ-БФ ГОСТ, 10 л.",антисептик универсальный prosept хмф бф гост
107,"Герметик акриловый цвет сосна, ф/п 600мл (12 штук )",герметик акриловый цвет сосна п 12 штука
165,Гель эконом-класса для мытья посуды вручную. Без запахаCooky Е концентрированное средство / 5 л ПЭТ,гель эконом класс мытьё посуда вручную запах cooky концентрированный средство пэт


In [432]:
dealer_products['product_name'].head()

0           Средство универсальное Prosept Universal Spray, 500мл
1        Концентрат Prosept Multipower для мытья полов, цитрус 1л
2    Средство для чистки люстр Prosept Universal Anti-dust, 500мл
3             Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05
4     Средство моющее для бани и сауны Prosept Multipower Wood 1л
Name: product_name, dtype: object

In [433]:
%%time
# аналогичным образом обработаем столбец product_name
dealer_products['dealer_name'] = dealer_products['product_name'].apply(clean_text)
dealer_products['dealer_name'] = dealer_products['dealer_name'].str.replace(pattern, '')
dealer_products['dealer_name'].head()

CPU times: total: 109 ms
Wall time: 100 ms


0              средство универсальное prosept universal spray 
1        концентрат prosept multipower для мытья полов цитрус 
2       средство для чистки люстр prosept universal anti dust 
3            удалитель ржавчины prosept rust remover 0  023 05
4    средство моющее для бани и сауны prosept multipower wood 
Name: dealer_name, dtype: object

In [434]:
%%time
dealer_products['dealer_name'] = dealer_products['dealer_name'].apply(preprocess_text)
dealer_products['dealer_name'].head()

CPU times: total: 16.4 s
Wall time: 16.4 s


0        средство универсальный prosept universal spray
1        концентрат prosept multipower мытьё пол цитрус
2    средство чистка люстра prosept universal anti dust
3        удалитель ржавчина prosept rust remover 023 05
4      средство мыть баня сауна prosept multipower wood
Name: dealer_name, dtype: object

In [435]:
dealer_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128 entries, 0 to 2127
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_key   2128 non-null   object 
 1   price         2128 non-null   float64
 2   product_url   2103 non-null   object 
 3   product_name  2128 non-null   object 
 4   date          2128 non-null   object 
 5   dealer_id     2128 non-null   int64  
 6   measures      2128 non-null   int32  
 7   name_length   2128 non-null   int32  
 8   dealer_name   2128 non-null   object 
dtypes: float64(1), int32(2), int64(1), object(5)
memory usage: 133.1+ KB


## Векторизация текста и поиск мэтчей

### Вариант 1: SBERT

In [53]:
%%time
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = list(products['marketing_name'])
query = dealer_products['dealer_name'][:10]
query_embedding = model.encode(query)
names_embeddings = model.encode(sentences)
cos_sim = util.cos_sim(query_embedding, names_embeddings)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.4490, 0.4633, 0.3566,  ..., 0.3623, 0.3242, 0.4982],
        [0.6617, 0.6051, 0.6739,  ..., 0.2962, 0.7048, 0.3938],
        [0.4186, 0.3653, 0.3867,  ..., 0.2638, 0.2587, 0.3311],
        ...,
        [0.6638, 0.6109, 0.5556,  ..., 0.4243, 0.4630, 0.5164],
        [0.6620, 0.7438, 0.6658,  ..., 0.4282, 0.4828, 0.4275],
        [0.4922, 0.6588, 0.5088,  ..., 0.3900, 0.4632, 0.4568]])
CPU times: total: 8min 36s
Wall time: 5min 10s


In [56]:
x = util.semantic_search(query_embedding, names_embeddings, top_k = 3)
x

[[{'corpus_id': 485, 'score': 0.9670079350471497},
  {'corpus_id': 340, 'score': 0.9448267817497253},
  {'corpus_id': 338, 'score': 0.9215435981750488}],
 [{'corpus_id': 173, 'score': 0.9255020022392273},
  {'corpus_id': 63, 'score': 0.9208823442459106},
  {'corpus_id': 15, 'score': 0.9057798385620117}],
 [{'corpus_id': 210, 'score': 0.9638019800186157},
  {'corpus_id': 207, 'score': 0.8129492402076721},
  {'corpus_id': 212, 'score': 0.8082730770111084}],
 [{'corpus_id': 281, 'score': 0.9451259970664978},
  {'corpus_id': 463, 'score': 0.9176199436187744},
  {'corpus_id': 250, 'score': 0.9176199436187744}],
 [{'corpus_id': 247, 'score': 0.9970699548721313},
  {'corpus_id': 197, 'score': 0.8693958520889282},
  {'corpus_id': 242, 'score': 0.8263242840766907}],
 [{'corpus_id': 438, 'score': 0.912359356880188},
  {'corpus_id': 462, 'score': 0.912359356880188},
  {'corpus_id': 400, 'score': 0.7782755494117737}],
 [{'corpus_id': 245, 'score': 0.9999998807907104},
  {'corpus_id': 246, 'score':

In [60]:
for i in range(len(x)):
    print('===================')
    print(f"Запрос: {dealer_products['product_name'][i]}")
    print('===================')
    for k in range(3):
        #print(sentences[x[i][k]['corpus_id']])
        print(f"Название продукта: {products['name_1c'][x[i][k]['corpus_id']]}, оценка: {x[i][k]['score']}")

Запрос: Средство универсальное Prosept Universal Spray, 500мл
Название продукта: Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л., оценка: 0.9670079350471497
Название продукта: Профессиональное чистящее средство PROSEPT Universal Spray, 5 л., оценка: 0.9448267817497253
Название продукта: Чистящая пена с антистатическим эффектом PROSEPT Universal Spray, 0.4 л., оценка: 0.9215435981750488
Запрос: Концентрат Prosept Multipower для мытья полов, цитрус 1л
Название продукта: Концентрат для мытья полов и стен Multipower "Цитрус" концентрат / 0,8 л, оценка: 0.9255020022392273
Название продукта: Средство для мытья полов PROSEPT Multipower "Цитрус", 2 штуки*0.8 л., оценка: 0.9208823442459106
Название продукта: Средство для мытья полов и стенMultipower "Полевые цветы"концентрат  1:10 – 1:120 / 0,8 л, оценка: 0.9057798385620117
Запрос: Средство для чистки люстр Prosept Universal Anti-dust, 500мл
Название продукта: Несмываемое средство для очистки люстр PROSEPT Universal Anti-dust, 0

### Вариант2: 

In [555]:
#Создадим словарь,где ключом являются id продукта, а значением - название
#marketing_name = pd.Series(products['marketing_name'].values, index=products['id']).to_dict()
marketing_name = pd.Series(products['name_1c'].values, index=products['id']).to_dict()
marketing_name

{245: 'Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.',
 3: 'Антигололед - 32 PROSEPTготовый состав / 12 кг',
 443: 'Герметик акриловый цвет сосна, ф/п 600мл',
 147: 'Кондиционер для белья "Королевский Ирис" Prosept Crystal Rinser, 2 л.',
 502: 'Герметик акриловой  цвет Белый, 7 кг',
 220: 'Грунт бетоноконтакт, для гладких поверхностей перед нанесением штукатурок, шпатлевок и плиточного клея, готовый состав, 6 кг',
 385: 'Грунт бетоноконтакт, для гладких поверхностей перед нанесением штукатурок, шпатлевок и плиточного клея, готовый состав, 12 кг',
 114: 'Средство для удаления технических масел, смазочных материалов и нефтепродуктов PROSEPT Duty Oil, 5 л.',
 505: 'Антисептик универсальный суперсильный',
 32: 'Профессиональное средство для мытья светлых полов с отбеливающим эффектом PROSEPT Multipower White, 1 л.',
 241: 'Отбеливатель для древесины PROSEPT 50, концентрат, 5 л.',
 339: 'Антисептик лессирующийзащитно-декоративныйPROSEPT BiO LASUR / ти

In [557]:
#Создадим словарь,где ключом являются product_key дилера, а значением - название у дилера
#dealer_name = pd.Series(dealer_products['dealer_name'].values, index=dealer_products['product_key']).to_dict()
dealer_name = pd.Series(dealer_products['product_name'].values, index=dealer_products['product_key']).to_dict()
dealer_name

{'546227': 'Средство универсальное Prosept Universal Spray, 500мл',
 '546408': 'Концентрат Prosept Multipower для мытья полов, цитрус 1л',
 '546234': 'Средство для чистки люстр Prosept Universal Anti-dust, 500мл',
 '651258': 'Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05',
 '546355': 'Средство моющее для бани и сауны Prosept Multipower Wood 1л',
 '831859': 'Пропитка PROSEPT Aquaisol для камня, концентрат  1:2  1л',
 '546406': 'Средство для мытья плитки и керамогранита Prosept Multipower Kerama 1л',
 '831858': 'Пропитка PROSEPT Aquaisol для камня, концентрат  1:2   5л',
 '857015': 'Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.',
 '651265': 'Соль для посудомоечных машин PROSEPT Splash 1,5кг 280-15',
 '546257': 'Средство для мытья стекол и зеркал Prosept Optic Crystal, 500мл',
 '831854': 'Антисептик PROSEPT Eco Ultra невымываемый, коричневый, готовый состав 5л',
 '831857': 'Очиститель фасадов  PROSEPT Salt Cleaner, концентрат 1:2  1л',
 '382757': 'Пропитка-антисепти

In [229]:
# создадим матрицу m * n, где m - названия товара у заказчика, n - название у дилера
# match_df = pd.DataFrame(index = products['marketing_name'], columns = dealer_products['dealer_name'])
# match_df.head()

In [236]:
%%time
rows = products['marketing_name'].values
columns = dealer_products['dealer_name'].values
market_names = model.encode(rows)
dealer_names = model.encode(columns)

CPU times: total: 44min 53s
Wall time: 28min 28s


In [640]:
# дополним векторы названий данными об объёме/весе и длине строки
X1 = pd.DataFrame(market_names)
X1['measures'] = products['measures']
X1['name_length'] = products['name_length']

X2 = pd.DataFrame(dealer_names)
X2['measures'] = dealer_products['measures']
X2['name_length'] = dealer_products['name_length']

# отмасштабируем данные
numeric = ['measures', 'name_length']
scaler = MinMaxScaler()

scaler.fit(X1[numeric])
X1[numeric] = scaler.transform(X1[numeric])

scaler.fit(X2[numeric])
X2[numeric] = scaler.transform(X2[numeric])

In [643]:
X1.values

array([[ 0.05087752,  0.02291129, -0.09482078, ...,  0.00537573,
         0.00125   ,  0.54205607],
       [ 0.05998654,  0.10665844, -0.10486306, ..., -0.08424861,
         0.015     ,  0.17757009],
       [ 0.04590375,  0.07513438, -0.02946339, ..., -0.06039172,
         0.75      ,  0.12149533],
       ...,
       [-0.02971146,  0.05097613, -0.08217428, ..., -0.06301438,
         0.625     ,  0.30841121],
       [ 0.02896112,  0.03088606, -0.04799785, ..., -0.01971812,
         0.00625   ,  0.64485981],
       [-0.0104496 , -0.00825288, -0.02729456, ..., -0.02191848,
         0.09375   ,  0.64485981]])

In [446]:
# преобразуем данные в разряженную матрицу
X1_sparse = sparse.csr_matrix(X1.values)
X2_sparse = sparse.csr_matrix(X2.values)

In [473]:
#создадим матрицу соответствий названий
# data = pairwise_distances(X1_sparse, X2_sparse, metric = 'cosine')
# match_df = pd.DataFrame(index = products['name_1c'], columns = dealer_products['product_name'], data=data)
# match_df

product_name,"Средство универсальное Prosept Universal Spray, 500мл","Концентрат Prosept Multipower для мытья полов, цитрус 1л","Средство для чистки люстр Prosept Universal Anti-dust, 500мл","Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",Средство моющее для бани и сауны Prosept Multipower Wood 1л,"Пропитка PROSEPT Aquaisol для камня, концентрат 1:2 1л",Средство для мытья плитки и керамогранита Prosept Multipower Kerama 1л,"Пропитка PROSEPT Aquaisol для камня, концентрат 1:2 5л","Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.","Соль для посудомоечных машин PROSEPT Splash 1,5кг 280-15",...,"Средство концентрированное Prosept Bath Fungi для удаления плесени с дезинфицирующим эффектом, 500м","Удалитель плесени PROSEPT FUNGI CLEAN готовый состав 0,5л 019-05",Удалитель плесени 1:1 PROSEPT FUNGI CLEAN 1л 019-1,Краска-грунт фасадная PROSEPT Proff 3 в 1 для плит OSB 7 кг,Краска-грунт фасадная PROSEPT OSB Proff 3 в 1 для плит 1 кг,"Средство для удаления клейкой ленты, клея, наклеек 0,4л PROSEPT Duty Universal готовый состав","Герметик акриловый межшовный для деревянных домов, конструкций, изделий PROSEPT цвет тик, 3кг",Отбеливатель для древесины PROSEPT ECO 50 готовый состав 1 кг,Строительный антисептик PROSEPT 5.6 кг,"Герметик акриловый межшовный для деревянных конструкций, цвет ""Орех"" готовый состав 0,6 кг"
name_1c,,,,,,,,,,,,,,,,,,,,,
"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",0.525660,0.284818,0.532529,0.503510,0.489485,0.279362,0.309163,0.281527,0.275534,0.423679,...,0.430051,0.460767,0.564164,0.288143,0.289222,0.248537,0.215241,0.349515,0.224306,0.276587
Антигололед - 32 PROSEPTготовый состав / 12 кг,0.560147,0.381485,0.637776,0.447813,0.455762,0.387475,0.410547,0.376488,0.259238,0.332167,...,0.634026,0.511985,0.628946,0.356017,0.360861,0.336887,0.342966,0.341740,0.245755,0.363178
"Герметик акриловый цвет сосна, ф/п 600мл",0.457911,0.464317,0.441935,0.640006,0.621270,0.555630,0.594424,0.550157,0.475082,0.584349,...,0.482266,0.686088,0.717669,0.520045,0.525210,0.571346,0.374714,0.558411,0.479488,0.353212
"Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.",0.574598,0.594740,0.587836,0.405599,0.538682,0.485907,0.557935,0.478518,0.470435,0.508878,...,0.445806,0.364293,0.439839,0.547805,0.543948,0.506583,0.491472,0.538804,0.562081,0.525653
"Герметик акриловой цвет Белый, 7 кг",0.690491,0.382901,0.681848,0.568341,0.562311,0.500556,0.552354,0.493002,0.341071,0.531714,...,0.688660,0.649289,0.667769,0.461822,0.461989,0.499891,0.261951,0.528142,0.395601,0.229891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Средство для уборки помещений после пожара с дезинфицирующим эффектом PROSEPT Duty Black, 5 л.",0.531094,0.381535,0.577497,0.481558,0.487780,0.412100,0.386984,0.401926,0.359473,0.429141,...,0.417821,0.439213,0.569356,0.323459,0.324414,0.231776,0.309503,0.425637,0.393183,0.371904
"Гель для стирки шерсти, шелка и деликатных тканей PROSEPT Crystal Delicate, 1 л.",0.584435,0.523937,0.584050,0.491858,0.475356,0.403682,0.526274,0.394117,0.337840,0.469074,...,0.501109,0.454698,0.530689,0.466207,0.466814,0.466184,0.418921,0.530718,0.472942,0.456524
"Cредство для чистки гриля и духовок Cooky Grill Gel, 500 мл.",0.438259,0.681072,0.501243,0.580009,0.623781,0.589700,0.668888,0.580362,0.576080,0.607312,...,0.447321,0.599842,0.650669,0.604880,0.608801,0.619620,0.561998,0.599259,0.662497,0.519037


In [560]:
# match_df.iloc[:, 1].sort_values(ascending=True)[:5]

name_1c
Концентрат для мытья полов и стен Multipower "Цитрус" концентрат / 0,8 л                     0.064343
Средство для мытья полов PROSEPT Multipower "Цитрус", 2 штуки*0.8 л.                         0.068875
Средство для мытья полов и стенMultipower "Полевые цветы"концентрат  1:10 – 1:120 / 0,8 л    0.087667
Средство для мытья полов PROSEPT Multipower "Полевые цветы", 2 штуки*0.8 л.                  0.090853
Набор 2 шт Концентрат для мытья полов и стен  Multipower "Полевые цветы" 0,8 л               0.113525
Name: Концентрат Prosept Multipower для мытья полов, цитрус 1л, dtype: float64

In [674]:
data = pairwise_distances(X1_sparse, X2_sparse, metric = 'cosine')
match_df = pd.DataFrame(index = products['id'], 
                        columns = dealer_products['product_key']+ '_' + pd.Series(range(dealer_products.shape[0])).astype(str), 
                        data=data)
match_df

,546227_0,546408_1,546234_2,651258_3,546355_4,831859_5,546406_6,831858_7,857015_8,651265_9,...,546235_2118,651256_2119,651257_2120,856988_2121,856956_2122,651260_2123,860509000_2124,900996549_2125,530170161_2126,528911039_2127
id,,,,,,,,,,,,,,,,,,,,,
245,0.525660,0.284818,0.532529,0.503510,0.489485,0.279362,0.309163,0.281527,0.275534,0.423679,...,0.430051,0.460767,0.564164,0.288143,0.289222,0.248537,0.215241,0.349515,0.224306,0.276587
3,0.560147,0.381485,0.637776,0.447813,0.455762,0.387475,0.410547,0.376488,0.259238,0.332167,...,0.634026,0.511985,0.628946,0.356017,0.360861,0.336887,0.342966,0.341740,0.245755,0.363178
443,0.457911,0.464317,0.441935,0.640006,0.621270,0.555630,0.594424,0.550157,0.475082,0.584349,...,0.482266,0.686088,0.717669,0.520045,0.525210,0.571346,0.374714,0.558411,0.479488,0.353212
147,0.574598,0.594740,0.587836,0.405599,0.538682,0.485907,0.557935,0.478518,0.470435,0.508878,...,0.445806,0.364293,0.439839,0.547805,0.543948,0.506583,0.491472,0.538804,0.562081,0.525653
502,0.690491,0.382901,0.681848,0.568341,0.562311,0.500556,0.552354,0.493002,0.341071,0.531714,...,0.688660,0.649289,0.667769,0.461822,0.461989,0.499891,0.261951,0.528142,0.395601,0.229891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.531094,0.381535,0.577497,0.481558,0.487780,0.412100,0.386984,0.401926,0.359473,0.429141,...,0.417821,0.439213,0.569356,0.323459,0.324414,0.231776,0.309503,0.425637,0.393183,0.371904
160,0.584435,0.523937,0.584050,0.491858,0.475356,0.403682,0.526274,0.394117,0.337840,0.469074,...,0.501109,0.454698,0.530689,0.466207,0.466814,0.466184,0.418921,0.530718,0.472942,0.456524
74,0.438259,0.681072,0.501243,0.580009,0.623781,0.589700,0.668888,0.580362,0.576080,0.607312,...,0.447321,0.599842,0.650669,0.604880,0.608801,0.619620,0.561998,0.599259,0.662497,0.519037


In [671]:
match_df.iloc[:, 1].sort_values()[:5]

id
45     0.387291
470    0.397950
471    0.462924
43     0.473618
435    0.520758
Name: 546408_1, dtype: float64

In [652]:
match[match['key'] == '546408']['product_id']

3    38
Name: product_id, dtype: int64

In [676]:
top = 10
match_df = match_df[list(set(match_df.columns))]

match_df.columns = match_df.columns.astype(str)

matches = []

for col in set(match_df.columns):
        
    top_cands = match_df.loc[:, col].sort_values(ascending=True)[:top].index.tolist()
    product_key = ''.join(col.split('_')[:-1])
    
    # print(product_key)
    
    if  match.loc[match['key'] == product_key].shape[0] == 0:
        matches.append(0)
        continue
        
    match_id = match.loc[match['key'] == product_key, 'product_id'].values[0]
    if match_id in top_cands:
        matches.append(1)
    else:
        matches.append(0)
        
np.mean(matches)

0.6273496240601504

In [537]:
top5 = match_df.iloc[:, 1].sort_values(ascending=True)[:5]
#for name in top5.index:
    #print(name)
top5.index[0]

'Концентрат для мытья полов и стен Multipower "Цитрус" концентрат / 0,8 л'

In [494]:
for k, v in dealer_name.items():
    if v == match_df.iloc[:,0].name:
        print(k)

546227


In [493]:
for k, v in marketing_name.items():
    for i in top5.index:
        if v == i:
            print(k)

15
69
14
13
12


### Вариант 3

In [545]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from scipy import sparse
from collections import defaultdict

# def get_text_length(x):
#     return np.array([len(t) for t in x]).reshape(-1, 1)

In [ ]:
# x_train = np.array(products['marketing_name'])
# x_test = np.array(dealer_products['dealer_name'])
# добавить длину как признак в матрицу

# tv = TfidfVectorizer(min_df = 0.05, max_df = 0.5, ngram_range = (1,2))
# X = tv.fit_transform(products['marketing_name'])
# vocab = tv.get_feature_names()
# X1 = pd.DataFrame(X.toarray(), columns = vocab)
# X1['len'] = products['name_length']
# X_sparse = sparse.csr_matrix(X1.values)

In [542]:
%%time
corpus = pd.concat([products['marketing_name'], dealer_products['dealer_name']], axis = 0)
count_tf_idf = TfidfVectorizer()
corpus_vect = count_tf_idf.fit(corpus)
df_1 = count_tf_idf.transform(products['marketing_name'])
df_2 = count_tf_idf.transform(dealer_products['dealer_name'])

TfidfVectorizer()

In [544]:
df_1 = df_1.todense() #df_1.toarray()
df_2 = df_2.todense() #df_2.toarray()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [546]:
matr = pd.DataFrame(data = pairwise_distances(df_1, df_2, 'cosine'), 
             index = products['id'], 
             columns = dealer_products['product_name'])
matr

product_name,"Средство универсальное Prosept Universal Spray, 500мл","Концентрат Prosept Multipower для мытья полов, цитрус 1л","Средство для чистки люстр Prosept Universal Anti-dust, 500мл","Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",Средство моющее для бани и сауны Prosept Multipower Wood 1л,"Пропитка PROSEPT Aquaisol для камня, концентрат 1:2 1л",Средство для мытья плитки и керамогранита Prosept Multipower Kerama 1л,"Пропитка PROSEPT Aquaisol для камня, концентрат 1:2 5л","Шпаклевка выравнивающая акриловая PROSEPT Plastix белая, 1 кг.","Соль для посудомоечных машин PROSEPT Splash 1,5кг 280-15",...,"Средство концентрированное Prosept Bath Fungi для удаления плесени с дезинфицирующим эффектом, 500м","Удалитель плесени PROSEPT FUNGI CLEAN готовый состав 0,5л 019-05",Удалитель плесени 1:1 PROSEPT FUNGI CLEAN 1л 019-1,Краска-грунт фасадная PROSEPT Proff 3 в 1 для плит OSB 7 кг,Краска-грунт фасадная PROSEPT OSB Proff 3 в 1 для плит 1 кг,"Средство для удаления клейкой ленты, клея, наклеек 0,4л PROSEPT Duty Universal готовый состав","Герметик акриловый межшовный для деревянных домов, конструкций, изделий PROSEPT цвет тик, 3кг",Отбеливатель для древесины PROSEPT ECO 50 готовый состав 1 кг,Строительный антисептик PROSEPT 5.6 кг,"Герметик акриловый межшовный для деревянных конструкций, цвет ""Орех"" готовый состав 0,6 кг"
name_1c,,,,,,,,,,,,,,,,,,,,,
"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",0.972574,0.886899,0.982325,0.984503,0.980499,0.980375,0.983193,0.980161,0.983840,0.986106,...,0.983062,0.984547,1.0,0.982109,0.982109,0.984800,0.856754,0.980552,0.845305,0.858872
Антигололед - 32 PROSEPTготовый состав / 12 кг,0.977400,0.980738,0.985436,0.987230,0.983931,0.983828,0.986150,0.983652,0.986684,0.988551,...,0.986042,0.834961,1.0,0.985257,0.985257,0.837666,0.987406,0.889950,0.970139,0.833868
"Герметик акриловый цвет сосна, ф/п 600мл",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.878874,1.000000,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.624717,1.000000,1.000000,0.586104
"Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.",0.979409,0.982449,0.986730,0.988365,0.985358,0.985265,0.987381,0.985105,0.987867,0.989568,...,0.987282,0.988397,1.0,0.986567,0.986567,0.988588,0.988525,0.985398,0.972792,1.000000
"Герметик акриловой цвет Белый, 7 кг",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.864511,1.000000,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.785961,1.000000,1.000000,0.763939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Средство для уборки помещений после пожара с дезинфицирующим эффектом PROSEPT Duty Black, 5 л.",0.935493,0.984991,0.958428,0.990050,0.954132,0.987399,0.960467,0.987262,0.989624,0.991079,...,0.711686,0.990078,1.0,0.988512,0.988512,0.893747,0.990186,0.987513,0.976732,1.000000
"Гель для стирки шерсти, шелка и деликатных тканей PROSEPT Crystal Delicate, 1 л.",0.982812,0.985350,0.988923,0.990288,0.987778,0.987700,0.989466,0.987567,0.989872,0.991292,...,0.989384,0.990315,1.0,0.988787,0.988787,0.990474,0.990421,0.987811,0.977289,1.000000
"Cредство для чистки гриля и духовок Cooky Grill Gel, 500 мл.",1.000000,1.000000,0.906694,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [213]:
#for column in (range(len(matr.columns))):
# matr.iloc[:, column].idxmin()
# matr.iloc[:, 100].argmin()

44

In [554]:
import scipy.sparse

pipe = Pipeline([('count', CountVectorizer(ngram_range= (1,2))),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe['count'].transform(corpus).toarray()
X = pipe['tfid'].idf_
X = pipe.transform(corpus)
X

<2621x3812 sparse matrix of type '<class 'numpy.float64'>'
	with 34114 stored elements in Compressed Sparse Row format>